## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

План проекта

1. Исследование данных

2. Предобработка данных

3. Работа с данными в целях формулирования выводов
3.1. Категоризация данных
3.2. Ответы на вопросы проекта

## 1. Исследование данных 

In [48]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info() # информация о таблице
data.describe()# вывод описательной статистики по переменным

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выведена информация о данных.
Выведена описательная статистика по переменным для поиска пропусков данных и аномалий.
В данных есть пропуски, об этом свидетельствует количество значений в переменных days_employed и total_income.
Присутствуют отрицательные значения в переменных days_employed и children.
Присутствуют неправдоподобно большие значения в переменной days_employed.

In [49]:
data['days_employed'] = abs(data['days_employed']) # избавляемся от отрицательных значений в days_employed
data['children'] = abs(data['children'])# избавляемся от отрицательных значений в переменной children
print('Минимальное количество детей:', data['children'].min())
print('Минимальное количество дней занятости:', data['days_employed'].min())

Минимальное количество детей: 0
Минимальное количество дней занятости: 24.14163324048118


Убираем отрицательные значения в переменных days_employed и children с помощью abs() - модуль.
Делаем проверку на наличие отрицательных значений.

In [50]:
def days_emp(type):
    days_emp_inc_type = data.loc[data.loc[:, 'income_type'] == type] # проверка длительности трудового стажа в зависимости от типов занятости
    ind_min = days_emp_inc_type['days_employed'].min()
    ind_max = days_emp_inc_type['days_employed'].max()
    print('Максимальное значение дней занятости для типа занятости', type,':', ind_max)
    print('Минимальное значение дней занятости для типа занятости', type,':', ind_min)
days_emp('пенсионер')
days_emp('госслужащий')
days_emp('компаньон')
days_emp('сотрудник')

Максимальное значение дней занятости для типа занятости пенсионер : 401755.40047533
Минимальное значение дней занятости для типа занятости пенсионер : 328728.72060451825
Максимальное значение дней занятости для типа занятости госслужащий : 15193.032201443106
Минимальное значение дней занятости для типа занятости госслужащий : 39.95417043834918
Максимальное значение дней занятости для типа занятости компаньон : 17615.563265627912
Минимальное значение дней занятости для типа занятости компаньон : 30.19533715555962
Максимальное значение дней занятости для типа занятости сотрудник : 18388.949900568383
Минимальное значение дней занятости для типа занятости сотрудник : 24.14163324048118


Проверяем к какому типу занятости относятся аномально высокие значения переменной days_employed.
Убеждаемся, что это пенсионеры. Исправления не требуются, т.к. трудовой стаж пенсионера для банка значения не имеет.

In [51]:
print(data['education'].value_counts())
print(data['family_status'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Обнаруживаем, что в значениях переменных education и family_status присутствуют прописные буквы.

In [52]:
data['education'] = data['education'].str.lower()# делаем буквы строчными
print(data['education'].value_counts())
data['family_status'] = data['family_status'].str.lower()# делаем буквы строчными
print(data['family_status'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Делаем все буквы в переменных education и family_status строчными, что позволит упростить обработку данных и уменьшает количество значений переменной education.

In [53]:
print(data['gender'].value_counts())
print(data.groupby('gender')['total_income'].mean())# расчет среднего дохода в зависимости от пола клиента

F      14236
M       7288
XNA        1
Name: gender, dtype: int64
gender
F      154097.529734
M      193169.652274
XNA    203905.157261
Name: total_income, dtype: float64


Обнаруживаем в переменной gender значение XNA. Рассчитываем средние значения дохода для каждого пола для определения, к какому полу относится человек с ошибочным значением.

In [54]:
data['gender'] = data['gender'].replace('XNA', 'M')# присвоение мужского пола непонятному клиенту
print(data['gender'].value_counts())

F    14236
M     7289
Name: gender, dtype: int64


Т.к. значение дохода для человека со значением XNA в переменной gender ближе к среднему значению дохода для мужского пола, заменяем в переменной gender значение XNA на M (мужской пол).
Проверяем, произошла ли замена.

In [55]:
print(data['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Обнаруживаем аномалию: 20 детей у  76 клиентов. Скорее всего, произошла ошибка при вводе данных (допечатали лишний 0).

In [56]:
data['children'] = data['children'].replace(20, 2)# исправляем количество детей
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


В переменной children заменяем значения 20 на 2.
Проверяем, прошла ли замена.

### Вывод

В данных есть пропуски, об этом свидетельствует количество значений в переменных days_employed и total_income.

В переменной days_employed присутствуют отрицательные значения (скорее всего опечатка или тире; исправлено: значения взяты по модулю), нереально большие значения (как показало исследование нереальные значения показателя относятся к типу занятости "пенсионер"; исправления не требует, т.к. трудовой стаж пенсионера для банка значения не имеет).

В переменной children присутствуют отрицательные значения, скорее всего, минус рядом с числом - это опечатка (исправлено: значения взяты по модулю). У 76 клиентов по 20 детей - аномалия, скорее всего, припечатали лишний 0. Заменяем 20 на 2.

В переменной education много вариаций написания каждого уровня образования (вариации прописных и строчных букв). Исправлено: все буквы сделаны строчными.

В переменной gender есть 3-й пол XNA. Видимо, ошибка. Судя по уровню дохода (рассчитаны средние по мужчинам и женщинам) - это мужчина. Исправлено.

В переменной dob_years есть нули. 

В переменной family_status в названиях статусов присутствуют прописные буквы. Меняем все на строчные в целях минимизации ошибок.

## 2. Предобработка данных

### Обработка пропусков

In [57]:
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
data['days_employed'] = data.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))
d_e = data.isnull().sum()
print(d_e)# проверка на наличие пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Выявленные ранее пропущенные значения в переменных days_employed и total_income заполняем медианными значениями соответствующих показателей в зависимости от типа занятости (income_type).
Проверяем переменные на наличие пропусков.

In [58]:
print((data['dob_years'] == 0).value_counts()) # проверка на наличие 0 в переменной возраста
data = data[data.dob_years != 0] # исключение строк с возрастом 0
print((data['dob_years'] == 0).value_counts())  # проверка на наличие 0 в переменной возраста

False    21424
True       101
Name: dob_years, dtype: int64
False    21424
Name: dob_years, dtype: int64


Проверяем переменную dob_years на наличие нулей.
В переменной dob_years есть нули. 101 клиент не указал свой возраст. Т.к. не понятно, как их восстанавливать (четких критериев нет), указанные данные были удалены.
Проверяем переменную dob_years на наличие нулей. Нулей нет.

### Вывод

В переменных days_employed и total_income есть пустые значения. Скорее всего, они появились в связи с нежеланием клиентов раскрывать информацию о себе. Заполним пропуски средними значениями соответствующих переменных по полу и типу занятости клиентов.

В переменной dob_years есть нули. 101 клиент не указал свой возраст. Т.к. не понятно, как их восстанавливать (четких критериев нет), указанные данные будут удалены.

### Замена типа данных

In [59]:
data['total_income'] = data['total_income'].astype('int')# перевод переменной total_income из дробной в целочисленные
data['days_employed'] = data['days_employed'].astype('int') # перевод переменной days_employed из дробной в целочисленные
data.info()# проверка

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null int64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null int64
purpose             21424 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Произведен перевод переменных days_employed и total_income из float64 d int64. 
Пороведена проверка осуществился ли перевод корректно. Тип переменных стал int64.

### Обработка дубликатов

In [60]:
print(data.duplicated().sum())# проверка на дубли
data = data.drop_duplicates() # удаление дублей
print(data.duplicated().sum())# проверка на дубли

71
0


### Вывод

При тесте на дубли обнаружена 71 дублированная строка с помощью duplicated. Т.к. полное совпадение значений всех переменных маловероятно, то, видимо, это дублированные строки, которые надо удалить. Дубли удалены методом drop_duplicates(). Проведена проверка удалились ли данные.

### Лемматизация

In [208]:
from pymystem3 import Mystem
m = Mystem()
def lemma(text):
    lemmas = m.lemmatize(text['purpose'])
    lemmas.remove('\n')
    return lemmas

data['lemmas'] = data.apply(lemma, axis = 1) 
print(data['lemmas'].head())

lis = [] # расчет частот слов в леммах
for row in data['lemmas']: # создание строки со всеми словами из целей кредита
    lis += row
    
from collections import Counter
print(Counter(lis))# подсчет количества каждого слова из списка


0                 [покупка,  , жилье]
1       [приобретение,  , автомобиль]
2                 [покупка,  , жилье]
3    [дополнительный,  , образование]
4               [сыграть,  , свадьба]
Name: lemmas, dtype: object
Counter({' ': 33435, 'недвижимость': 6328, 'покупка': 5870, 'жилье': 4436, 'автомобиль': 4284, 'образование': 3995, 'с': 2904, 'операция': 2593, 'свадьба': 2310, 'свой': 2223, 'на': 2210, 'строительство': 1873, 'высокий': 1366, 'получение': 1309, 'коммерческий': 1306, 'для': 1286, 'жилой': 1224, 'сделка': 938, 'дополнительный': 902, 'заниматься': 900, 'проведение': 764, 'сыграть': 760, 'сдача': 649, 'семья': 637, 'собственный': 633, 'со': 627, 'ремонт': 605, 'подержанный': 484, 'подержать': 478, 'приобретение': 459, 'профильный': 435})


### Вывод

В переменной purpose цели кредитов указаны не системно. Одной цели может соответствовать несколько вариантов. Для того, чтобы систематизировать цели кредита нужна лемматизация. Загружаем библиотеку Mystem. Создаем функцию, которая выделяет слова в именительном падеже по каждой строке. Записываем результат в переменную lemmas.
Рассчитываем частоты упоминания каждого слова в целях кредита. Наиболее популярными словами, несущими смысл, стали:
недвижимость - 6328 упоминаний, покупка - 5870 упоминаний, жилье - 4436 упоминаний, автомобиль - 4284 упоминаний, образование- 3995 упоминаний, операция - 2593 упоминаний, свадьба - 2310 упоминаний.
Слова "покупка" и "операция" могут относиться к разным целям, поэтому они отбрасываются.
Соответственно основными целями кредита признаются: недвижимость, жилье, автомобиль, образование, свадьба.
При этом жилье и недвижимость целесообразно объединить в одну категорию "недвижимость".

## 3. Работа с данными в целях формулирования выводов

### Категоризация данных

In [195]:
def unif_purpose(lem): # категоризация целей кредита
    word = lem['lemmas']
    if 'автомобиль' in word:
        return 'приобретение автомобиля'
    if 'жилье' in word:
        return 'приобретение недвижимости'
    if 'недвижимость' in word:
        return 'приобретение недвижимости'
    if 'свадьба' in word:
        return 'организация свадьбы'
    return 'получение образования'
data['un_purpose'] = data.apply(unif_purpose, axis = 1)
print(data[['lemmas','un_purpose']].head(10))

                                  lemmas                 un_purpose
0                    [покупка,  , жилье]  приобретение недвижимости
1          [приобретение,  , автомобиль]    приобретение автомобиля
2                    [покупка,  , жилье]  приобретение недвижимости
3       [дополнительный,  , образование]      получение образования
4                  [сыграть,  , свадьба]        организация свадьбы
5                    [покупка,  , жилье]  приобретение недвижимости
6             [операция,  , с,  , жилье]  приобретение недвижимости
7                          [образование]      получение образования
8        [на,  , проведение,  , свадьба]        организация свадьбы
9  [покупка,  , жилье,  , для,  , семья]  приобретение недвижимости


Категоризируем данные по целям кредита в соответствии с выявленными выше обобщенными целями.
Категоризация осуществляется по ключевым словам (автомобиль, жилье, недвижимость, свадьба, образование).
Проверяем разноску лемм по категориям.

In [197]:
def cat_age(years): # категоризация возраста клиентов
    age = years['dob_years']
    if age < 36:
        return 'молодежь'
    if age < 56:
        return 'зрелые'
    return 'пожилые'
data['cat_age'] = data.apply(cat_age, axis = 1)
print(data[['dob_years','cat_age']].head(10))

   dob_years   cat_age
0         42    зрелые
1         36    зрелые
2         33  молодежь
3         32  молодежь
4         53    зрелые
5         27  молодежь
6         43    зрелые
7         50    зрелые
8         35  молодежь
9         41    зрелые


Категоризируем данные по возрасту клиентов.
Категоризация осуществляется по возрасту (35 и моложе - молодежь, 35-55 - зрелые, старше 55 - пожилые).
Проверяем разноску возрастов клиентов по категориям.

In [202]:
# print(data['total_income'].quantile(.7))
def cat_income(money): # категоризация доходов клиентов
    income = money['total_income']
    if income <= data['total_income'].quantile(.35):
        return 'низкие доходы'
    if income <= data['total_income'].quantile(.7):
        return 'средние доходы'
    return 'высокие доходы'
data['cat_income'] = data.apply(cat_income, axis = 1)
print(data[['total_income','cat_income']].head(10))

   total_income      cat_income
0        253875  высокие доходы
1        112080   низкие доходы
2        145885  средние доходы
3        267628  высокие доходы
4        158616  средние доходы
5        255763  высокие доходы
6        240525  высокие доходы
7        135823  средние доходы
8         95856   низкие доходы
9        144425  средние доходы


Категоризируем данные по уровню дохода клиентов.
Категоризация осуществляется по квантилям доходов (35% квантиля и ниже - низкие доходы, 35%-70% квантиля - средние доходы, более 70% квантиля - высокие доходы).
Проверяем разноску возрастов клиентов по доходам.

### Вывод

Лемматизированные значения из столбца lemmas с помощью условий наличия ключевых слов в строках по каждому клиенту приводим к обобщенным категориям целей кредита. Записываем категории по каждой строке в переменную un_purpose.

Целесообразно выделить категории клиентов по возрасту и уровню дохода.С помощью условий выделяем категории в переменные  cat_age и cat_income.

### Ответы на вопросы проекта

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [203]:
print('Расчет вероятностей возврата кредита в срок клиентами в зависимости от наличия детей')
data_pivot_c = data.pivot_table(index=['children'], columns = 'debt',values = 'gender', aggfunc='count', margins=True)
data_pivot_c['propab'] = data_pivot_c[1] / data_pivot_c['All']*100
print(data_pivot_c)

Расчет вероятностей возврата кредита в срок клиентами в зависимости от наличия детей
debt            0       1    All    propab
children                                  
0         12964.0  1058.0  14022  7.545286
1          4397.0   442.0   4839  9.134119
2          1912.0   202.0   2114  9.555345
3           301.0    27.0    328  8.231707
4            37.0     4.0     41  9.756098
5             9.0     NaN      9       NaN
All       19620.0  1733.0  21353  8.115956


### Вывод

Да, есть. Вероятность возникновения просрочки по кредиту у клиента с детьми - колеблется от 8,2% до 9,6%, у клиента без детей - 7,5%. Таким образом, банку выгоднее предоставлять кредиты клиентам без детей.
Если рассмотреть вероятности возникновения просрочки по кредиту у клиентов в зависимости от количества детей, то наиболее выгодными клиентами для банка останутся клиенты без детей (вероятность просрочки - 7,5%), на втором месте оказались клиенты с 3-мя детьми (вероятность - 8,2%), на третьем месте - клиенты с 1-м ребенком (вероятность - 9,1%).
Клиенты с 5-ю детьми, несмотря на нулевую вероятность просрочки кредита, не могут рассматриваться, как репрезентативная выборка, т.к. их очень мало для анализа (9 чел.)
Вероятность рассчитана как отношение клиентов, имеющих просрочку по кредиту, из соответствующей группы к общему количеству клиентов из указанной группы. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [204]:
print('Расчет вероятностей возврата кредита в срок клиентами в зависимости от семейного положения')
data_pivot = data.pivot_table(index=['family_status'], columns = 'debt', values = 'children', aggfunc='count', margins=True)
data_pivot['propab'] = data_pivot[1] / data_pivot['All']*100
print(data_pivot)

Расчет вероятностей возврата кредита в срок клиентами в зависимости от семейного положения
debt                       0     1    All    propab
family_status                                      
в разводе               1100    85   1185  7.172996
вдовец / вдова           892    62    954  6.498952
гражданский брак        3744   386   4130  9.346247
женат / замужем        11363   927  12290  7.542718
не женат / не замужем   2521   273   2794  9.770938
All                    19620  1733  21353  8.115956


### Вывод

Да, есть. Наименьшая вероятность допустить просрочку по кредиту наблюдается у вдов/вдовцов (6,5%), на втором месте разведенные (7,2%), на третьем месте женатые/замужние (7,5%). Самые невыгодные для банка клиенты - не вструпившие в брак (вероятность просрочки - 9,8%).
Вероятность рассчитана как отношение клиентов, имеющих просрочку по кредиту, из соответствующей группы к общему количеству клиентов из указанной группы. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [205]:
print('Расчет вероятностей возврата кредита в срок клиентами в зависимости от уровня дохода')
data_pivot = data.pivot_table(index=['cat_income'], columns = 'debt', values = 'children', aggfunc='count', margins=True)
data_pivot['propab'] = data_pivot[1] / data_pivot['All']*100
print(data_pivot)

Расчет вероятностей возврата кредита в срок клиентами в зависимости от уровня дохода
debt                0     1    All    propab
cat_income                                  
высокие доходы   5927   479   6406  7.477365
низкие доходы    6862   612   7474  8.188386
средние доходы   6831   642   7473  8.590927
All             19620  1733  21353  8.115956


### Вывод

Да, есть. Наименьшая вероятность допустить просрочку по кредиту наблюдается у клиентов с высокими доходами (7,5%), на втором месте клиенты с низкими доходами (8,2%), Самые невыгодные для банка клиенты - со средними доходами (вероятность просрочки - 8,6%).
Вероятность рассчитана как отношение клиентов, имеющих просрочку по кредиту, из соответствующей группы к общему количеству клиентов из указанной группы. 

- Как разные цели кредита влияют на его возврат в срок?

In [206]:
print('Расчет вероятностей возврата кредита в срок клиентами в зависимости от целей кредита')
data_pivot = data.pivot_table(index=['un_purpose'], columns = 'debt', values = 'children', aggfunc='count', margins=True)
data_pivot['propab'] = data_pivot[1] / data_pivot['All']*100
print(data_pivot)

Расчет вероятностей возврата кредита в срок клиентами в зависимости от целей кредита
debt                           0     1    All    propab
un_purpose                                             
организация свадьбы         2126   184   2310  7.965368
получение образования       3625   370   3995  9.261577
приобретение автомобиля     3884   400   4284  9.337068
приобретение недвижимости   9985   779  10764  7.237087
All                        19620  1733  21353  8.115956


### Вывод

С точки зрения целей кредита, наиболее выгодными клиентами для банка являются лица, желающие взять кредит на ипотеку (вероятность просрочки 7,2%), что обусловлено серьезностью намерений людей, собирающихся купить недвижимость. Как правило, перед тем как решиться на приобретение недвижимости, люди все просчитывают и серьезно готовятся к этому шагу. 

На втором месте по выгоде для банка кредиты на организацию свадьбы (вероятность просрочки 7,9%), что обусловлено помощью супруга при возврате такого кредита.

Наименее выгодные для банка цели кредитов: получение образования (9,2%) и приобретение автомобиля (9,3%), что обусловлено молодостью и неопытностью клиентов, берущих такие кредиты.

### Шаг 4. Общий вывод

Исходя из полученных данных, наиболее выгодным клиентом для банка, с точки зрения возврата кредита, является вдовец/вдова без детей с высокими доходами, берущий кредит на приобретение недвижимости. 

Наименее выгодным клиентом будет клиент со средними доходами, не состоящий в браке, имеющий детей и берущий кредит на приобретение автомобиля.  